In [ ]:
1fafrom google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Instalando la libreria necesaria

In [ ]:
!pip install abydos

In [ ]:
import pandas as pd
import numpy as np
import string
import re           # Regular Expression
import itertools    # Colecciones de datos

import scipy.spatial.distance as scipydis

import time

from abydos.distance import * 
from abydos.phonetic import *

import abydos.distance as dd

In [ ]:
pd.read_csv('/content/drive/MyDrive/Proyecto/Datos/Extraidos/2020_COFEPRIS_SS_Listado_de_medicamentos_de_Referencia.csv').head()

,Unnamed: 0,Número,Fármaco(s),Forma Farmacéutica \n(consideración de uso),Concentración,Fracción \n(Art. 226 \nLGS),Denominación,Registro Sanitario,Titular,Situación \nactual,Indicación terapéutica
0,0,1,"ABACAVIR, Sulfato de",Tableta,300 mg,IV,ZIAGENAVIR,088M99 SSA,"GLAXOSMITHKLINE MÉXICO, S.A. DE C.V.",Prorrogado,Antirretroviral.
1,1,2,"ABACAVIR, Sulfato de",Solución,2 g / 100 mL,IV,ZIAGENAVIR,087M99 SSA,"GLAXOSMITHKLINE MÉXICO, S.A. DE C.V.",Prorrogado,Antirretroviral.
2,2,3,"ABACAVIR, Sulfato de / \nLAMIVUDINA",Tableta,600 mg / 300 mg,IV,KIVEXA,479M2005 SSA,"GLAXOSMITHKLINE MÉXICO, S.A. DE C.V.",Prorrogado,Antirretroviral.
3,3,4,"ABIRATERONA, Acetato de",Tableta,250 mg,IV,ZYTIGA,241M2012 SSA,"JANSSEN-CILAG, S.A. DE C.V.",En evaluación,En el tratamiento de pacientes con cáncer de p...
4,4,5,ACARBOSA,Tableta,50 mg y 100 mg,IV,GLUCOBAY,458M93 SSA,"BAYER DE MÉXICO, S.A. DE C.V.",Prorrogado,Para el tratamiento de la diabetes mellitus ti...


In [ ]:
# Extraemos los datos
denominacion = pd.read_csv('/content/drive/MyDrive/Proyecto/Datos/Extraidos/2020_COFEPRIS_SS_Listado_de_medicamentos_de_Referencia.csv', usecols=['Denominación'])
type(denominacion)

pandas.core.frame.DataFrame

In [ ]:
def clean_text(text):
    # remove numbers
    text_no_n = re.sub('\n', '', text) # Saltos de lines
    text_nonum = re.sub(r'\d+', '', text_no_n) # Digitos
    
    # remove punctuations and convert characters to lower case
    text_nopunct = "".join([char.upper() for char in text_nonum if char not in string.punctuation]) 
    # substitute multiple whitespace with single whitespace
    # Also, removes leading and trailing whitespaces
    text_no_doublespace = re.sub('\s+', ' ', text_nopunct).strip()
    return text_no_doublespace

In [ ]:
# Limpieza del texto
limpieza = [clean_text(text) for text in denominacion['Denominación'].values]

In [ ]:
# Cantidad de datos
np.unique(limpieza).shape

(1385,)

In [ ]:
# Extremos nombre unicos, por si existen duplicados
nombresMed = np.asarray(np.unique(limpieza))
type(nombresMed)

numpy.ndarray

In [ ]:
# Tamaño del nombre (palabras) y su frecuencia
# Cuantos medicamentos existen de n-palabras

np.asarray(np.unique([len(str(i).split(' ')) for i in nombresMed], return_counts=True)).T

array([[   1, 1035],
       [   2,  324],
       [   3,   24],
       [   4,    1],
       [   6,    1]])

    [Tamaño, Frecuencia]
       [   1, 1035],
       [   2,  324],
       [   3,   24],
       [   4,    1],
       [   6,    1]
    

In [ ]:
# Solo se analizan los nombre con tamaño 1 o 2
size = 1
medicamentos = [nombre for nombre in nombresMed if len(str(nombre).split(' ')) <= 1]
print(f'# Medicamentos: {len(medicamentos)}')

# Medicamentos: 1035


In [ ]:
# Lista de pares de medicamentos, Todos contra todos, ejemplo.
list(itertools.combinations(medicamentos[:3],2))

[('ABELCET', 'ABILIFY'), ('ABELCET', 'ABRIXONE'), ('ABILIFY', 'ABRIXONE')]

In [ ]:
%%time
# Genera las combinaciones de todos contra todos, en tamaño 2
combinaciones = list(itertools.combinations(medicamentos[:],2))

CPU times: user 57.1 ms, sys: 34.1 ms, total: 91.2 ms
Wall time: 92.5 ms


In [ ]:
# Primeras cobinaciones
combinaciones[:10]

[('ABELCET', 'ABILIFY'),
 ('ABELCET', 'ABRIXONE'),
 ('ABELCET', 'ACCOLATE'),
 ('ABELCET', 'ACLASTA'),
 ('ABELCET', 'ACON'),
 ('ABELCET', 'ACROMICINA'),
 ('ABELCET', 'ACTIKERALL'),
 ('ABELCET', 'ACTRON'),
 ('ABELCET', 'ACULAREN'),
 ('ABELCET', 'ACUPRIL')]

In [ ]:
len(combinaciones)

535095

In [ ]:
dc = {

# Como se escribe el medicamento
'Levenshtein': dd.Levenshtein(),
'OSA': dd.Levenshtein(mode='osa'),
'FlexMetric': dd.FlexMetric(),
'BISIM': dd.BISIM(),
'PhoneticEditDistance': dd.PhoneticEditDistance(),
'Strcmp95': dd.Strcmp95(),
'Prefix': dd.Prefix(),
'Suffix': dd.Suffix(),

# Como se escucha el medicamento
'PD_Soundex': dd.PhoneticDistance(transforms=[Soundex], metric=Levenshtein),
'PD_PSHPSoundexFirst': dd.PhoneticDistance(transforms=[PSHPSoundexFirst], metric=Levenshtein),
'PD_PSHPSoundexLast': dd.PhoneticDistance(transforms=[PSHPSoundexLast], metric=Levenshtein),
'PD_MRA': dd.PhoneticDistance(transforms=[MRA], metric=Levenshtein),
'PD_Davidson': dd.PhoneticDistance(transforms=[Davidson], metric=Levenshtein),
'PD_Dolby': dd.PhoneticDistance(transforms=[Dolby], metric=Levenshtein),
'PD_ParmarKumbharana': dd.PhoneticDistance(transforms=[ParmarKumbharana], metric=Levenshtein),
'PD_PhoneticSpanish': dd.PhoneticDistance(transforms=[PhoneticSpanish], metric=Levenshtein),
'PD_SpanishMetaphone': dd.PhoneticDistance(transforms=[SpanishMetaphone], metric=Levenshtein),
'PD_ParmarKumbharana': dd.PhoneticDistance(transforms=[ParmarKumbharana], metric=Levenshtein)
}

In [ ]:
%%time

for name, dist in dc.items():
    print('Calculando distancia: ', name)

    distancias = [dist.dist(i,j) for i,j in combinaciones]
    mtx_distancias = pd.DataFrame(data=scipydis.squareform(distancias), index=medicamentos, columns=medicamentos)
    mtx_distancias.to_pickle(f'/content/drive/MyDrive/Proyecto/Datos/Extraidos/Distancias/{name}.pkl')
    
    print()

Calculando distancia:  Levenshtein

Calculando distancia:  OSA

Calculando distancia:  FlexMetric

Calculando distancia:  BISIM

Calculando distancia:  PhoneticEditDistance

Calculando distancia:  Strcmp95

Calculando distancia:  Prefix

Calculando distancia:  Suffix

Calculando distancia:  PD_Soundex

Calculando distancia:  PD_PSHPSoundexFirst

Calculando distancia:  PD_PSHPSoundexLast

Calculando distancia:  PD_MRA

Calculando distancia:  PD_Davidson

Calculando distancia:  PD_Dolby

Calculando distancia:  PD_ParmarKumbharana

Calculando distancia:  PD_PhoneticSpanish

Calculando distancia:  PD_SpanishMetaphone

CPU times: user 21min 11s, sys: 962 ms, total: 21min 12s
Wall time: 21min 16s


In [ ]:
mtx_distancias

,ABELCET,ABILIFY,ABRIXONE,ACCOLATE,ACLASTA,ACON,ACROMICINA,ACTIKERALL,ACTRON,ACULAREN,ACUPRIL,ADALAT,ADEPSIQUE,ADIMOD,ADVANTAN,ADVIL,AEROFLUX,AFINITOR,AFRIN,AFUMIX,AGGRENOX,AKATINOL,AKINETON,ALBOTHYL,ALDACTONE,ALDARA,ALDOMET,ALEPSAL,ALGITRIN,ALIMTA,ALIN,ALKARELIEF,ALKASELTZER,ALKERAN,ALLEGRA,ALMAX,ALMETEC,ALMETECCO,ALMOGRAN,ALOSOL,...,XALATAN,XARELTO,XELODA,XENICAL,XUZAL,XYLODERM,YASMIN,YODOLACTINA,YONDELIS,YUNIR,YUREDOL,YURELAX,ZACAFEMYL,ZACTOS,ZADITEN,ZALDIAR,ZANIDIP,ZELMAC,ZEMPLAR,ZENHALE,ZENTEL,ZESTORETIC,ZIAGENAVIR,ZIENT,ZINNAT,ZINTREPID,ZOCOR,ZOELY,ZOFRAN,ZOLADEX,ZOLINZA,ZOMETA,ZOMIG,ZOVIRAX,ZYLOPRIM,ZYMAR,ZYPLO,ZYPREXA,ZYTIGA,ZYVOXAM
ABELCET,0.0,0.4,0.6,0.4,0.4,0.80,0.666667,0.833333,0.8,0.6,0.8,0.40,0.8,0.80,0.666667,0.60,0.8,0.8,0.80,0.80,0.833333,0.8,0.8,0.666667,0.666667,0.60,0.6,0.8,0.666667,0.40,0.6,0.833333,0.666667,0.8,0.80,0.60,0.6,0.6,0.833333,0.60,...,0.80,0.6,0.8,1.00,0.80,0.833333,1.00,0.666667,1.0,1.00,1.00,0.80,1.000000,1.00,1.00,0.80,1.00,0.80,1.0,1.00,1.00,0.833333,1.0,0.80,0.80,1.000000,1.00,0.80,1.00,0.80,0.80,0.80,1.00,0.80,0.833333,1.0,1.0,1.0,1.0,1.0
ABILIFY,0.4,0.0,0.6,0.6,0.6,0.80,0.833333,0.666667,0.8,0.6,0.8,0.60,0.8,0.80,0.666667,0.60,0.8,0.8,0.80,0.80,0.833333,0.8,0.8,0.500000,0.833333,0.60,0.8,0.8,0.833333,0.60,0.6,0.666667,0.833333,0.8,0.80,0.60,0.8,0.8,0.833333,0.60,...,0.80,0.8,0.8,1.00,0.80,0.833333,1.00,0.833333,1.0,1.00,1.00,0.80,0.833333,1.00,1.00,0.80,1.00,0.80,1.0,1.00,1.00,1.000000,1.0,1.00,1.00,1.000000,1.00,0.60,1.00,0.80,0.80,1.00,1.00,0.80,0.833333,1.0,1.0,1.0,1.0,1.0
ABRIXONE,0.6,0.6,0.0,0.8,0.8,0.60,0.500000,0.833333,0.6,0.6,0.8,0.80,0.8,0.80,0.500000,0.80,0.8,0.8,0.40,0.60,0.666667,0.8,0.6,0.666667,0.666667,0.80,0.8,0.8,0.666667,0.80,0.6,0.666667,0.833333,0.6,0.80,0.60,0.8,0.8,0.666667,0.80,...,0.80,0.8,1.0,1.00,1.00,1.000000,0.80,0.833333,1.0,0.80,0.80,0.80,1.000000,1.00,0.80,1.00,1.00,1.00,1.0,1.00,1.00,0.833333,1.0,1.00,1.00,0.833333,0.80,1.00,0.60,0.80,1.00,1.00,1.00,0.40,1.000000,1.0,1.0,0.8,1.0,0.8
ACCOLATE,0.4,0.6,0.8,0.0,0.4,0.75,0.833333,0.833333,0.8,0.6,0.8,0.25,0.8,0.75,0.666667,0.75,0.6,0.6,0.75,0.75,0.833333,0.8,0.6,0.666667,0.666667,0.75,0.6,0.8,0.666667,0.50,0.5,0.666667,0.500000,0.8,0.75,0.75,0.6,0.6,0.833333,0.75,...,0.50,0.5,0.5,1.00,0.75,0.666667,1.00,0.666667,0.8,1.00,1.00,0.75,1.000000,1.00,1.00,1.00,1.00,1.00,0.8,1.00,1.00,0.833333,1.0,0.75,0.75,1.000000,1.00,0.75,1.00,1.00,1.00,0.75,1.00,1.00,0.833333,1.0,1.0,1.0,1.0,1.0
ACLASTA,0.4,0.6,0.8,0.4,0.0,0.60,0.666667,0.666667,0.6,0.4,0.6,0.40,0.6,0.80,0.666667,0.80,0.8,0.8,0.80,0.80,0.833333,0.6,0.6,0.666667,0.666667,0.60,0.6,0.6,0.666667,0.40,0.6,0.666667,0.500000,0.8,0.80,0.60,0.6,0.6,0.833333,0.40,...,0.80,0.6,0.8,0.80,0.80,0.833333,1.00,0.666667,0.8,1.00,1.00,0.80,0.833333,0.60,1.00,0.80,1.00,0.80,1.0,1.00,1.00,0.833333,1.0,0.80,0.80,1.000000,0.80,0.80,1.00,0.80,0.80,0.80,1.00,1.00,0.833333,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZYMAR,1.0,1.0,1.0,1.0,1.0,1.00,0.833333,0.833333,0.8,0.8,0.8,1.00,1.0,0.75,1.000000,1.00,1.0,0.8,1.00,0.75,0.833333,1.0,1.0,1.000000,1.000000,0.75,0.8,1.0,0.833333,0.75,1.0,0.833333,1.000000,0.8,0.50,0.75,0.8,0.8,0.666667,1.00,...,1.00,1.0,1.0,1.00,1.00,0.666667,0.75,1.000000,1.0,0.50,1.00,1.00,0.666667,0.75,0.75,0.50,0.75,0.50,0.6,0.75,0.75,0.666667,0.6,0.75,0.75,0.666667,0.50,0.75,0.75,0.75,0.75,0.50,0.50,0.75,0.500000,0.0,0.5,0.4,0.5,0.6
ZYPLO,1.0,1.0,1.0,1.0,1.0,1.00,1.000000,1.000000,1.0,1.0,0.6,1.00,0.8,1.00,1.000000,0.75,0.8,1.0,1.00,1.00,1.000000,0.8,1.0,1.000000,1.000000,1.00,1.0,0.6,1.000000,1.00,1.0,0.833333,0.833333,1.0,0.75,1.00,1.0,1.0,1.000000,0.75,...,1.00,1.0,1.0,0.75,0.75,0.833333,1.00,0.833333,0.8,0.75,0.75,0.75,0.666667,0.75,0.75,0.75,0.75,0.75,0.4,0.50,0.

---
---
---
___